<a href="https://colab.research.google.com/github/mertege/FMCW-Data-Classification-/blob/main/Spectrogram_CI4R_Datasets_Multi_Class_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CI4R datasetinde sadece 77 GHz kullanılarak Neural Network koşturuldu. 
# 5 defa run edildi ve değerler aşağıda paylaşıldı.
# Mean test accuracy is 0.79, max test accuracy is 0.80, min test accuracy is 0.78,  std of test accuracy is 0.01.
# Time elapsed through all process: 12688.21, sec

In [ ]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate
from tensorflow.keras import regularizers
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K 
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 77 GHz
bending_77Ghz                 = np.load('/content/drive/MyDrive/bending_77Ghz.npy')
label_bending_77Ghz           = np.zeros((bending_77Ghz.shape[0],1))
label_bending_77Ghz[:]        = 1
crawling_77Ghz                = np.load('/content/drive/MyDrive/crawling_77Ghz.npy')
label_crawling_77Ghz          = np.zeros((crawling_77Ghz.shape[0],1))
label_crawling_77Ghz[:]       = 2
kneeling_77Ghz                = np.load('/content/drive/MyDrive/kneeling_77Ghz.npy')
label_kneeling_77Ghz          = np.zeros((kneeling_77Ghz.shape[0],1))
label_kneeling_77Ghz[:]       = 3
limping_with_RL_Stiff_77Ghz   = np.load('/content/drive/MyDrive/limping_with_RL_Stiff_77Ghz.npy')
label_limping_with_RL_Stiff_77Ghz           = np.zeros((limping_with_RL_Stiff_77Ghz.shape[0],1))
label_limping_with_RL_Stiff_77Ghz[:]        = 4
picking_up_an_object_77Ghz    = np.load('/content/drive/MyDrive/picking_up_an_object_77Ghz.npy')
label_picking_up_an_object_77Ghz          = np.zeros((picking_up_an_object_77Ghz.shape[0],1))
label_picking_up_an_object_77Ghz[:]       = 5
scissors_gait_77Ghz           = np.load('/content/drive/MyDrive/scissors_gait_77Ghz.npy')
label_scissors_gait_77Ghz          = np.zeros((scissors_gait_77Ghz.shape[0],1))
label_scissors_gait_77Ghz[:]       = 6
short_steps_77Ghz             = np.load('/content/drive/MyDrive/short_steps_77Ghz.npy')
label_short_steps_77Ghz          = np.zeros((short_steps_77Ghz.shape[0],1))
label_short_steps_77Ghz[:]       = 7
sitting_77Ghz                 = np.load('/content/drive/MyDrive/sitting_77Ghz.npy')
label_sitting_77Ghz          = np.zeros((sitting_77Ghz.shape[0],1))
label_sitting_77Ghz[:]       = 8
walking_away_from_Radar_77Ghz = np.load('/content/drive/MyDrive/walking_away_from_Radar_77Ghz.npy')
label_walking_away_from_Radar_77Ghz         = np.zeros((walking_away_from_Radar_77Ghz.shape[0],1))
label_walking_away_from_Radar_77Ghz[:]       = 9
Walking_on_both_toes_77Ghz    = np.load('/content/drive/MyDrive/Walking_on_both_toes_77Ghz.npy')
label_Walking_on_both_toes_77Ghz         = np.zeros((Walking_on_both_toes_77Ghz.shape[0],1))
label_Walking_on_both_toes_77Ghz[:]       = 10
Walking_towards_radar_77Ghz   = np.load('/content/drive/MyDrive/Walking_towards_radar_77Ghz.npy')
label_Walking_towards_radar_77Ghz          = np.zeros((Walking_towards_radar_77Ghz.shape[0],1))
label_Walking_towards_radar_77Ghz[:]       = 11
# # Xethru
# bending_Xethru = np.load('/content/drive/MyDrive/bending_Xethru.npy')
# crawling_Xethru = np.load('/content/drive/MyDrive/crawling_Xethru.npy')
# kneeling_Xethru = np.load('/content/drive/MyDrive/kneeling_Xethru.npy')
# limping_with_RL_Stiff_Xethru = np.load('/content/drive/MyDrive/limping_with_RL_Stiff_Xethru.npy')
# picking_up_an_object_Xethru = np.load('/content/drive/MyDrive/picking_up_an_object_Xethru.npy')
# scissors_gait_Xethru = np.load('/content/drive/MyDrive/scissors_gait_Xethru.npy')
# short_steps_Xethru = np.load('/content/drive/MyDrive/short_steps_Xethru.npy')
# sitting_Xethru = np.load('/content/drive/MyDrive/sitting_Xethru.npy')
# walking_away_from_Radar_Xethru = np.load('/content/drive/MyDrive/walking_away_from_Radar_Xethru.npy')
# Walking_on_both_toes_Xethru = np.load('/content/drive/MyDrive/Walking_on_both_toes_Xethru.npy')
# Walking_towards_radar_Xethru = np.load('/content/drive/MyDrive/Walking_towards_radar_Xethru.npy')
# # 24 GHz
# bending_24GHz = np.load('/content/drive/MyDrive/bending_24GHz.npy')
# crawling_24GHz = np.load('/content/drive/MyDrive/crawling_24GHz.npy')
# kneeling_24GHz = np.load('/content/drive/MyDrive/kneeling_24GHz.npy')
# limping_with_RL_Stiff_24GHz = np.load('/content/drive/MyDrive/limping_with_RL_Stiff_24GHz.npy')
# picking_up_an_object_24GHz = np.load('/content/drive/MyDrive/picking_up_an_object_24GHz.npy')
# scissors_gait_24GHz = np.load('/content/drive/MyDrive/scissors_gait_24GHz.npy')
# short_steps_24GHz = np.load('/content/drive/MyDrive/short_steps_24GHz.npy')
# sitting_24GHz = np.load('/content/drive/MyDrive/sitting_24GHz.npy')
# walking_away_from_Radar_24GHz = np.load('/content/drive/MyDrive/walking_away_from_Radar_24GHz.npy')
# Walking_on_both_toes_24GHz = np.load('/content/drive/MyDrive/Walking_on_both_toes_24GHz.npy')
# Walking_towards_radar_24GHz = np.load('/content/drive/MyDrive/Walking_towards_radar_24GHz.npy')

In [ ]:
spectrogram_concat = np.concatenate((bending_77Ghz,crawling_77Ghz),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,kneeling_77Ghz               ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,limping_with_RL_Stiff_77Ghz  ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,picking_up_an_object_77Ghz   ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,scissors_gait_77Ghz          ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,short_steps_77Ghz            ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,sitting_77Ghz                ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,walking_away_from_Radar_77Ghz),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,Walking_on_both_toes_77Ghz   ),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,Walking_towards_radar_77Ghz  ),axis=0)

label_concat = np.concatenate((label_bending_77Ghz,label_crawling_77Ghz),axis=0)
label_concat = np.concatenate((label_concat,label_kneeling_77Ghz               ),axis=0)
label_concat = np.concatenate((label_concat,label_limping_with_RL_Stiff_77Ghz  ),axis=0)
label_concat = np.concatenate((label_concat,label_picking_up_an_object_77Ghz   ),axis=0)
label_concat = np.concatenate((label_concat,label_scissors_gait_77Ghz          ),axis=0)
label_concat = np.concatenate((label_concat,label_short_steps_77Ghz            ),axis=0)
label_concat = np.concatenate((label_concat,label_sitting_77Ghz                ),axis=0)
label_concat = np.concatenate((label_concat,label_walking_away_from_Radar_77Ghz),axis=0)
label_concat = np.concatenate((label_concat,label_Walking_on_both_toes_77Ghz   ),axis=0)
label_concat = np.concatenate((label_concat,label_Walking_towards_radar_77Ghz  ),axis=0)

In [ ]:
# One-Hot Encode Label
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label_concat)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded)
spectrogram_concat_label = onehot_encoded_labels

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, spectrogram_concat.shape[0]), spectrogram_concat.shape[0]) # split validation data
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]

In [ ]:
range_doppler_concat_shuffle = spectrogram_concat_shuffle
range_doppler_concat_label = label_concat
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]

In [ ]:
# # Concat range-doppler data
# range_doppler_concat = np.concatenate((range_doppler_fast_resized,range_doppler_slow_resized),axis=0)
# range_doppler_concat = np.concatenate((range_doppler_concat,range_doppler_slow_pocket_resized),axis=0)
# range_doppler_concat = range_doppler_concat[:,:,:,np.newaxis] 
# range_doppler_concat_label = label_concat
# # Shuffle concat range doppler
# shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
# range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
# range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]
# # Concat range-doppler data
# spectrogram_concat = np.concatenate((spectrogram_fast_resized,spectrogram_slow_resized),axis=0)
# spectrogram_concat = np.concatenate((spectrogram_concat,spectrogram_slow_pocket_resized),axis=0)
# spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis] 
# spectrogram_concat_label = onehot_encoded_labels
# # Shuffle concat range doppler
# # shuffle_indx = random.sample(range(0, spectrogram_concat.shape[0]), spectrogram_concat.shape[0]) # split validation data
# spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
# spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]

In [ ]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
def mixup_augmentation(images,range_doppler_training_data, labels, repeat_of_mixup, alpha=0.2):
    batch_size = images.shape[0]
    concat_images = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],range_doppler_training_data\
                                            .shape[2],range_doppler_training_data.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))
    
    for ii in range(repeat_of_mixup):
      # shuffle train dataset
      shuffle_indx_1 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_1 = images[shuffle_indx_1,:,:,:]
      range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
      labels_shuffled_1 = labels[shuffle_indx_1,:]

      shuffle_indx_2 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_2 = images[shuffle_indx_2,:,:,:]
      range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
      labels_shuffled_2 = labels[shuffle_indx_2,:]

      # Sample lambda and reshape it to do the mixup
      ll = np.random.beta(alpha, alpha, (batch_size,1,1,1))
      x_l = np.reshape(ll, (batch_size,1,1,1))
      y_l = np.reshape(ll, (batch_size,1))
      
      # Perform mixup on both images and labels by combining a pair of images/labels
      images_mixup = images_shuffled_1 * x_l + images_shuffled_2 * (1 - x_l)
      images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)
      labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * (1 - y_l)
      concat_images[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup
      concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
      concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

    concat_images[repeat_of_mixup*batch_size:,:,:,:] = images
    concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
    concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images,concat_images_range_doppler, concat_label)

def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,range_doppler_concat_label_shuffle_train,\
                                       repeat_of_mixup, augmentation_enable):
  # ---------------- Parameters ----------------
  size_of_validation = 50
  # Split validation
  randomlist_for_validation_indx = random.sample(range(0, range_doppler_concat_shuffle_train.shape[0]), size_of_validation) # split validation data
  randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data
  # get validation data
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  spectrogram_validation_labels = range_doppler_concat_label_shuffle_train[randomlist_for_validation_indx,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  # get training data
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]

  (spectrogram_augmented_image,range_doppler_augmented_image,spectrograms_label)=\
   mixup_augmentation(spectrogram_training_data,range_doppler_training_data, spectrogram_training_labels, repeat_of_mixup, alpha=0.2)

  return (spectrogram_augmented_image,range_doppler_augmented_image,spectrograms_label,\
     spectrogram_validation_data,range_doppler_validation_data, spectrogram_validation_labels)



def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


In [ ]:
normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)


In [ ]:
t = time.time()
# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 128
dropout_prob_cnn = 0.1
dropout_prob_dense = 0.3
repeat_of_mixup = 15
number_of_repeat = 5
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del range_doppler_concat_shuffle_test
    del spectrogram_concat_shuffle_test
    del range_doppler_augmented_image
    del range_doppler_concat_shuffle_train
    del spectrogram_concat_shuffle_train
    del spectrogram_augmented_image
   
  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):   
    gc.collect()
    K.clear_session()
    
    # test data
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    range_doppler_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    #train data
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
      # ---------------- MixUp Augmentation ----------------
    (spectrogram_augmented_image,range_doppler_augmented_image,spectrogram_concat_label_shuffle_concat,\
     validation_spectrogram,validation_range_doppler, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,\
                                         spectrogram_concat_label_shuffle_train,\
                                         repeat_of_mixup, augmentation_enable)
    
    # ---------------- Neural Network Architecture ----------------

    def encoder_for_range_doppler(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(4, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.2)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Flatten()(x)
      return Model(input, x)


    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(128,kernel_regularizer='l2')(input)
      # x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(0.5)(x)
      x = Dense(dense_size,kernel_regularizer='l2')(x)
      # x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(11, activation="softmax")(x)
      return Model(input, x)
    input_shape = range_doppler_concat_shuffle.shape[1:]
    base_network_range_doppler = encoder_for_range_doppler(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_range_doppler(range_doppler_input)


    base_decoder_network = decoder_for_concat((processed_range_doppler.shape[1]))
    out = base_decoder_network(processed_range_doppler)

    model = Model(inputs=[range_doppler_input], outputs=[out]) 
    # print(model.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=35, verbose=0,restore_best_weights=True, mode='min')
    history = model.fit((range_doppler_augmented_image),(spectrogram_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler) , (spectrogram_validation_labels)))
    
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test],\
                                               [range_doppler_concat_label_shuffle_test],
                  batch_size=batch_size)
    # # ---------------- Get Test Results ----------------
    # y_test_predicted = model.predict((range_doppler_concat_shuffle_test), batch_size=batch_size)
    # # ----- Binarize y_test_predicted values -----
    # y_test_predicted_binary = np.zeros(y_test_predicted.shape[0])
    # for ii in range(y_test_predicted.shape[0]):
    #   if argmax(y_test_predicted[ii,:]) == 0:
    #     y_test_predicted_binary[ii] = 0
    #   elif argmax(y_test_predicted[ii,:]) == 1:
    #     y_test_predicted_binary[ii] = 1
    #   else:
    #     y_test_predicted_binary[ii] = 2
    # range_doppler_concat_label_shuffle_test1 = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]
    # test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(range_doppler_concat_label_shuffle_test1, y_test_predicted_binary, average='macro')

    test_accuracy_per_fold.append(test_accuracy)
    # f1_score_per_fold.append(test_f1_score)
    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  # f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
  print(test_accuracy_per_run)
  # print(f1_score_per_run)
print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))},\
min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}.')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Epoch 1/100
234/234 [==============================] - 16s 29ms/step - loss: 2.8491 - accuracy: 0.3277 - val_loss: 1.3528 - val_accuracy: 0.5400
Epoch 2/100
234/234 [==============================] - 6s 28ms/step - loss: 1.2499 - accuracy: 0.6919 - val_loss: 0.9919 - val_accuracy: 0.7400
Epoch 3/100
234/234 [==============================] - 6s 27ms/step - loss: 1.0228 - accuracy: 0.8129 - val_loss: 1.0261 - val_accuracy: 0.7600
Epoch 4/100
234/234 [==============================] - 6s 27ms/step - loss: 0.9135 - accuracy: 0.8705 - val_loss: 0.9541 - val_accuracy: 0.7400
Epoch 5/100
234/234 [==============================] - 6s 28ms/step - loss: 0.8480 - accuracy: 0.8970 - val_loss: 0.9945 - val_accuracy: 0.7600
Epoch 6/100
234/234 [==============================] - 6s 28ms/step - loss: 0.8145 - accuracy: 0.9101 - val_loss: 1.1805 - val_accuracy: 0.6800
Epoch 7/100
234/234 [==============================] - 6s 28ms/step - loss: 0.7756 - accuracy: 0.9229 - val_loss: 0.9944 - val_accuracy

In [ ]:
y_test_predicted = model.predict((range_doppler_concat_shuffle_test), batch_size=batch_size)

In [ ]:
print(spectrogram_validation_labels)

In [ ]:
print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}.')
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Mean test accuracy is 0.79, max test accuracy is 0.80, min test accuracy is 0.78,  std of test accuracy is 0.01.
Time elapsed through all process: 12688.21, sec
